In [1]:
import kagglehub

path = kagglehub.dataset_download("shahzaibshazoo/detect-ai-generated-faces-high-quality-dataset")

print("Path to dataset files:", path)

100%|██████████| 116M/116M [00:07<00:00, 16.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shahzaibshazoo/detect-ai-generated-faces-high-quality-dataset/versions/1


In [18]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/root/.cache/kagglehub/datasets/shahzaibshazoo/detect-ai-generated-faces-high-quality-dataset/versions/1/AI-face-detection-Dataset'

# Directories for the split datasets
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

for dir in [train_dir, val_dir, test_dir]:
    os.makedirs(dir, exist_ok=True)
    os.makedirs(os.path.join(dir, 'AI'), exist_ok=True)
    os.makedirs(os.path.join(dir, 'real'), exist_ok=True)

def split_data(source, train_dir, val_dir, test_dir, train_size=0.7, val_size=0.2):
    """
    Splits the data into train, validation, and test sets and moves the files to their respective directories.
    """
    all_files = [os.path.join(source, file) for file in os.listdir(source) if os.path.isfile(os.path.join(source, file))]

    train_files, temp_files = train_test_split(all_files, train_size=train_size, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=len(temp_files) - int(len(all_files) * val_size), random_state=42)

    def copy_files(files, target_dir):
        for file in files:
            shutil.copy(file, target_dir)

    copy_files(train_files, train_dir)
    copy_files(val_files, val_dir)
    copy_files(test_files, test_dir)

# Perform the split for 'AI' and 'real'
split_data(os.path.join(base_dir, 'AI'), os.path.join(train_dir, 'AI'), os.path.join(val_dir, 'AI'), os.path.join(test_dir, 'AI'))
split_data(os.path.join(base_dir, 'real'), os.path.join(train_dir, 'real'), os.path.join(val_dir, 'real'), os.path.join(test_dir, 'real'))

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    shuffle=False)  # Important for testing to ensure consistent order


Found 2241 images belonging to 2 classes.
Found 640 images belonging to 2 classes.
Found 322 images belonging to 2 classes.


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output
])

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
base_model.trainable = False  # Freeze the convolutional base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Set steps per epoch and validation steps so all data is used
steps_per_epoch = 1793 // 32
validation_steps = 96 // 32

# Fit the model on the data
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=2
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/2
56/56 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.8942 - loss: 0.2347 - val_accuracy: 0.9583 - val_loss: 0.0640
Epoch 2/2
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 398ms/step - accuracy: 0.9840 - loss: 0.0414 - val_accuracy: 0.9792 - val_loss: 0.0397


In [29]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),  # Adjust the target size to match the model's expected input
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


test_loss, test_accuracy = model.evaluate(test_generator)
print("Test accuracy:", test_accuracy)

# Save the model
model.save('ai_vs_real_model.keras')

Found 322 images belonging to 2 classes.
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9888 - loss: 0.0337
Test accuracy: 0.97826087474823


In [30]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt

model = load_model('ai_vs_real_model.keras')

In [1]:
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

def upload_and_predict():
    uploaded = files.upload()

    for fn in uploaded.keys():
        path = '/content/' + fn
        img = load_img(path, target_size=(64, 64))  # Adjusted to correct input size
        plt.imshow(img)
        plt.axis('off')
        plt.show()

        # Preprocess the image
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.

        # Make predictions
        prediction = model.predict(img_array)
        predicted_class = 'AI' if prediction[0][0] > 0.5 else 'Real'
        confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]
        print(f'The uploaded image is an {predicted_class} image with {confidence:.2%} confidence.')

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


    heatmap = make_gradcam_heatmap(img_array, model, 'conv2d_2')
    plt.matshow(heatmap)
    plt.title('Grad-CAM')
    plt.show()


# Call the function
upload_and_predict()


KeyboardInterrupt: 

OpenAI. "Development Assistance for AI Face Image Detector Code by ChatGPT, Large Language Model." OpenAI, 2025. https://chatgpt.com.
